<a href="https://colab.research.google.com/github/miriammazzeo95/BigData_and_Timeseries_in_Pyspark/blob/main/Pyspark_ConfigurationImportsFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Set Up Pyspark in Colab**


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
#https://stackoverflow.com/questions/55240940/error-while-installing-spark-on-google-colab
%cd /

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] =  "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

/


In [ ]:
from functools import *
import itertools 
from itertools import chain
import os
import subprocess
from os import listdir
from os.path import isfile, join
import sys, math

import datetime
from datetime import *

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.window import Window
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession

spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")

# **Functions**

In [ ]:
print('start file')
from functools import *
import itertools 
from itertools import chain
import os
import subprocess
from os import listdir
from os.path import isfile, join
import sys, math

import datetime
from datetime import *

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.window import Window
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession

spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")

start file


In [ ]:
def read_timeseries(dump_freq: str, year_month: tuple,  day=None):
    if day:
        data = spark.read.parquet(f"/feddl/landing/trefor/quarter_hourly/{dump_freq}/year={year_month[0]}/month={year_month[1]}/day={day}")
    else:
        data = spark.read.parquet(f"/feddl/landing/trefor/quarter_hourly/{dump_freq}/year={year_month[0]}/month={year_month[1]}")
    return data

####################################################################################################################################    

def df_filterByColumnValue(df, col_name: str, col_item: str):
    # filters dataframe rows on a specific string in a column
    df = df.filter(col(col_name) == col_item)
    
    return df

####################################################################################################################################

def forward_fill_int(df, col_name, order_col_name, fill_col_name):
    
    '''Forward fill a column by a column/set of columns (order_col).  
    Parameters:
    ------------
    df: Dataframe 
    col_name: String, column of  to be cleaned
    order_col: String, column names (part of df) selected to order the window
    fill_col_name: String, name of the new cleaned column
    Return:
    ---------
    df: Dataframe 
        Return df with the filled_cols. 
    +---------+-------------------+-------+
    |cookie_ID|               Time|User_ID|
    +---------+-------------------+-------+
    |        1|2015/12/01 00:15:00|   55.8|
    |        1|2015/12/01 00:30:00|   null|
    |        1|2015/12/01 01:00:00|   -7.0|
    |        1|2015/12/01 02:00:00|  29.23|
    |        1|2015/12/01 01:45:00|   32.0|
    |        1|2015/12/01 05:00:00|   24.5|
    |        2|2015/12/01 03:30:00|   39.0|
    |        2|2015/12/01 02:45:00|   52.0|
    |        2|2015/12/01 03:15:00|   53.0|
    |        2|2015/12/01 02:15:00|  250.0|
    +---------+-------------------+-------+
    '''
    
    #change column type to Double
    df = df.withColumn(col_name, df[col_name].cast(DoubleType()))
   
    col_var = df.agg({col_name : 'variance'}).first()[0]
    col_stdev =  col_var**(1/2)
    col_mean = df.agg(avg(col(col_name))).first()[0]

    # "value" and "tmp" are temporary columns created ton enable forward fill. 
    # mapping null values and outliers with zeroes
    df = df.withColumn('value', when((col(col_name) >= (col_mean + 2*col_stdev)) | (col(col_name) <= (col_mean - 2*col_stdev)) | col(col_name).isNull(), 0).otherwise(1)) 
    
    # Find cumulative sum of a the 'value' column
    df = df.withColumn('tmp', lit('tmp')) 
    windowval = (Window.partitionBy('tmp') 
                 .orderBy(df[order_col_name[0]].asc(), df[order_col_name[1]].asc())
                 .rangeBetween(Window.unboundedPreceding, 0)) 
    df = df.withColumn('cum_sum', sum('value').over(windowval).alias('cumsum'))
    df = df.drop('tmp') 
    df = df.drop('value')

    # forward filling 
    win = (Window.partitionBy('cum_sum').orderBy(order_col_name))
    df = df.withColumn(fill_col_name, collect_list(col_name).over(win)[0])

    df = df.drop('cum_sum') 

    
    return df   
 

####################################################################################################################################    

def forward_fill_str(df, col_name, order_col_name, fill_col_name):
    
    '''Forward fill a column by a column/set of columns (order_col).  
    Parameters:
    ------------
    df: Dataframe 
    col_name: String, column of strings that we want to 'clean' with forward fille
    order_col: tuple of String, column names (part of df) selected to order the window
    fill_col_name: String, name of the new cleaned column
    Return:
    ---------
    df: Dataframe 
        Return df with the filled_cols. 
    '''
    
    #change column type to String
    df = df.withColumn(col_name, df[col_name].cast("string"))
    
    # "value" and "tmp" are temporary columns created ton enable forward fill. 
    # mapping null values and outliers with zeroes
    df = df.withColumn('value', when((col(col_name) == "?") | (col(col_name) == " ") | (col(col_name) == "") | (col(col_name).contains("?")) | (col(col_name) == '?') | (col(col_name).isNull()) | (col(col_name).rlike('[\?]')), 0).otherwise(1)) 
    
    # Find cumulative sum of a the 'value' column
    df = df.withColumn('tmp', lit('tmp')) 
    windowval = (Window.partitionBy('tmp') 
                 .orderBy(df[order_col_name[0]].asc(), df[order_col_name[1]].asc())
                 .rangeBetween(Window.unboundedPreceding, 0)) 
    df = df.withColumn('cum_sum', sum('value').over(windowval).alias('cumsum'))
    df = df.drop('tmp') 
    df = df.drop('value')

    # forward filling 
    win = (Window.partitionBy('cum_sum').orderBy(order_col_name[0]))
    df = df.withColumn(fill_col_name, collect_list(col_name).over(win)[0])

    df = df.drop('cum_sum') 
    
    return df   

####################################################################################################################################   

def col_to_hour(df, column_name: str, new_column_name: str):
    """
    000 -> 0.00 -> 00:00:00, 630 -> 6.30 -> 06:00:00, 45 -> 0.45 -> 00:00:00, 12.15 -> 12:00:00
    """
    df = df.withColumn(column_name, col(column_name)/100)
    df = df.withColumn(column_name, col(column_name).cast(StringType()))
    df = df.withColumn(column_name, substring_index(col(column_name), ".", 1))
    df = df.withColumn(new_column_name, date_format(to_timestamp(col(column_name),"H"), "HH:mm:ss"))
    
    return df

####################################################################################################################################   

def df_mapCol(df, df_dict, new_col, key_col):
    def translate(mapping):
        def translate_(col):
            return mapping.get(col)
        return udf(translate_, StringType())
    
    df_remapped = df.withColumn(new_col, translate(df_dict)(key_col))
    return df_remapped

####################################################################################################################################   

def createStationCols(df):
    df = df.withColumn("MissingColumns", split("MissingColumns", ","))
    df = df.select(*df.columns[:-1], df.MissingColumns[0].alias('Station1'), df.MissingColumns[1].alias('Station2'), df.MissingColumns[2].alias('Station3'), df.MissingColumns[3].alias('Station4'))
    df = df.withColumn("Station1", regexp_replace("Station1", "\W+", ""))
    df = df.withColumn("Station4", regexp_replace("Station4", "\W+", ""))
    return df

####################################################################################################################################   

def merge_cols(df, column_name: str, columnToDrop_name: str, new_column_name: str, sep=' '):
    """
    merging two string columns in a new column with name column_name3, separeted by sep (a space by default)
    the new column is inserted in the df first place 
    """
    df=df.withColumn(new_column_name, concat_ws(sep, df[column_name], df[columnToDrop_name]))
       
    df=df.drop(columnToDrop_name)
    return df

####################################################################################################################################   

def dfCol_to_time(df, column_name: str, format = '%H.%M'):
    """
    takes a df, a column name and the time format of the column values 
    if time values are such as
    0, 630, 45... 
    the values are mapped as
    0 -> 0.00, 630 -> 6.30, 45 -> 0.45, 1215 -> 12.15
    and transformed in data-time format '%H.%M' such as 
    0.00 -> 00:00:00, 6.30 -> 06:30:00, 0.45 -> 00:45:00, 12.15 -> 12:15:00
    """
    # dividing each value in column_name by 100
    df = df.withColumn(column_name, df[column_name]/100)
    
    # increasing decimal precision to 2 decimals for each value in column_name
    df = df.withColumn(column_name, df[column_name].cast(DecimalType(10,2)))
    
    # converting int to str in column_name
    df = df.withColumn(column_name, df[column_name].cast(StringType()))
    
    # mapping with lambda expreassion each value to format in column_name to data-time type 
    #Source func: https://stackoverflow.com/questions/45031810/replace-substring-of-values-in-a-dataframe-in-pyspark
    func = UserDefinedFunction(lambda x: datetime.strptime(x, format), TimestampType())
    df = df.withColumn(column_name, func(col(column_name)))
    
    # changing data-time format to only time
    # Source: https://stackoverflow.com/questions/63691162/how-to-extract-time-from-timestamp-in-pyspark
    df = df.withColumn(column_name, date_format(column_name, 'HH:mm:ss'))
    
    return df
    
####################################################################################################################################   

def get_filePaths(shell_cmd):
    """
    takes in imput a string containing a shell path-reading command, for example: 'hdfs dfs -ls /feddl/landing/trefor/...'
    returns a list of str whith file paths contained in the path specified in input
    """
    
    paths = subprocess.run(shell_cmd, shell=True, universal_newlines = True, stdout = subprocess.PIPE).stdout.splitlines()
    paths = [p.split()[-1] for p in paths]
    paths = [p for p in paths if p[0:6] == '/feddl']
    paths = [p for p in paths if p[-1:] != 'S']
    
    return paths
    
# TEST - get all file path in Trefor landing zone
# all_trefor_days = get_filePaths('hdfs dfs -ls /feddl/landing/trefor/quarter_hourly/*/*/*/*/*/')

####################################################################################################################################  

def df_to_dict(path):
    # READ DF DICTIONARY (SAVED PREVIOUSLY) IN 'feddl/work/CDK/miriam/dictionary/'
    df_dict = spark.read.parquet(path)
    #df_dict.explain(True)
    df_dict = df_dict.withColumn("InstallationKabelskab", col('InstallationKabelskab').cast('int'))

    # SORTING ALPHABETICALLY
    df_dict = df_dict.sort(col("InstallationAdresse").asc())

    # CONVERT TO REAL DICTIONARY 
    dictionary = {row['InstallationAdresse']:row['InstallationKabelskab'] for row in df_dict.collect()}
    return dictionary

####################################################################################################################################  

def rename_columns(df, dict_col):
    """
    takes a df and a dictionary as {oldName:newName}
    """
    for col in df.schema.names:
        df = df.withColumnRenamed(col,dict_col[col])
    return df
    
def add_index_column(df, position = "last"):
    """
    add monotonous incresingly index (UID) as first or last column
    """
    original_columns = df.columns
    df_with_index = df.select("*").withColumn("UID", monotonically_increasing_id())
    
    if position == "last":
        return df_with_index
        
    elif position == "first":
        new_list_columns = ["UID"]
        new_list_columns.extend(original_columns)
        df_with_index = df_with_index.select(new_list_columns)
        return df_with_index
    else: 
        print ("not implemented, select first or last")
        
def remove_cols(df,drop_cols, multiple = True):
    """
    remove one or multiple columns from dataframe 
    """
    if multiple:
        df = df.drop(*drop_cols)
    else:
        #df = df.drop(col(f"{drop_cols}"))
        df = df.drop(col(drop_cols))
    return df


####################################################################################################################################   

def remove_file(file_path):
    """
    """
    
    subprocess.run(f'hdfs dfs -rm -r {file_path}', shell=True, universal_newlines = True, stdout = subprocess.PIPE)
